# Tent review 데이터 전처리

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit
# ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import os
import pandas as pd
import numpy as np
import re

## 데이터 로드

In [2]:
df = pd.read_csv('./tent-review.csv')

/home/ec2-user/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df[['star_score', 'review']]

,star_score,review
0,5.0,원래 원터치텐트만 구입하려고 마음먹은 사람입니다. 기존 패스트캠프 테라6 원터치 텐...
1,3.0,좋아요.한여름에 창이 4개가 아니라 고생했지많요..<br>확실히 두개라서 바람이 잘...
2,5.0,1.배송이 빨리왔어요^^<br>2.펼치고 접기 너무 쉬워요~<br> ㅡ사실 저희부...
3,5.0,구매하기전에 정말 고민 많이했어요~~ <br>아이와 캠핑가기 위해 설치가 편한 제품...
4,5.0,첫 캠핑이라 너무 설레서 <em>고민고민하다가 무엇을 사야하나 하고 </em>매장도...
...,...,...
107274,3.0,배송이 좀 느렸다는점.. 다음날 받은게 아니어서요... 택배사 문제인듯...
107275,4.0,가격대비 가성비 좋아요
107276,5.0,상품좋습니다 크기도생각보다큼
107277,4.0,색깔이 흰색인줄 알았는데 회색이더군요\n아직 펴보진 않았는데 일단 가볍긴하네요


In [93]:
df['review'][2]

'1.배송이 빨리왔어요^^<br>2.펼치고 접기 너무 쉬워요~<br>  ㅡ사실 저희부부 둘다 텐트를 펴고 접어본적이없어서 원터치나 쉽게 펴고접을수 있는것을 살려고 검색을 많이 했어요~~ 근데 원터치들은 기둥도없고 많이 흐느적 거리더라구요, 그러다가 오토6를 봤는데 육각형으로 다 지지대가 있어서 튼튼해요!! 바람이 불어도 튼튼해요!! 그럼에도 펴고 접기가 쉽더라고요~~ <br>신랑이 늦게와서 제가 집에서 먼저 한번 펼쳐봤는데, 여자인 제가 펴고 쉽더라구요^^<br>접을때도 설명서에 있는대로, 아래쪽 다리를 딱 꺽어서 접으니까 3단 우산접듯이 아주 쉽게 접어지고 우산처럼 싹 말아주면 끝이더라구요~<br>3.넓어요~<br>ㅡ시부모님과 바다에가서 처음으로 펴봤는데, 시부모님과 저희부부 그리고 아이까지 앉아서 텐트속에서 맛난 점심을 아주 여유롭게먹었답니다^^<br>코시국에 식당가기도 힘든데, 배달이나 도시락으로 텐트속에서 오손도손 <em>식사하니 눈치볼 필요없이 너무 좋았어요</em>~^^<br>4.간편하지만 캠핑용으로도 손색없을듯<br>ㅡ 간편한데 <em>내구성이 좋아서</em>, 겨울빼고 봄여름가을에 가벼운 캠핑용으로 좋을것같아요~~ <br>사실 저희는 여름에 아이랑 쉬는 용도로 삿는데 오토6를 계기로 의자랑 테이블도 사서 본격 캠핑족에 한번 들어가볼까 해요^^ <br><br>무튼, 저희는 5점만점에 5000000000점입니다^^'

### 패턴 확인

In [4]:
a = re.sub('<.+?>', '', df['review'][0])

In [5]:
a

'원래 원터치텐트만 구입하려고 마음먹은 사람입니다. 기존 패스트캠프 테라6 원터치 텐트 사용하다가 좀 작은 사이즈도 추가로 구입하고 싶어서 오토6로 하나 더 구입했습니다. 손에 익었을때  펴는데 2분정도 걸려요. 접는데도 그정도는 걸리는거 같구요.가방에 넣었을때 부피가 작고 가벼워서 좋습니다. 미니멀캠핑이라 구성품에 있는 번들팩으로도 사용이 충분하고, 추가 팩 구입 할 필요 없을듯해요. 나무데크에서 주로 야영하니 오징어팩만 쓸때고 있구요. 천고는 테라6가 160이어서 엄청 편했는데 이번에 산 오토6도 그것보다는 낮지만 전혀 불편감 없네요. 레인플라이(레인커버), 전용카페트, 그라운드시트, 익스텐션월  추가 구매했습니다. 타프 가지고 있는분은 익스텐션월 추가 구매 불필요할거 같구요. 딱 비 피하고 짐 안젖을 정도의 공간만 나와요 ㅎ..미니멀캠핑하시는분은 익스텐션월까지 추가 구입하시고, 타프 사용 안하시면 될 거 같구요. (만원더 보태서 크기가 좀더 큰 착탈익스텐션 월 사시는것도 괜찮을거 같구요.) 텐트 내부는 어른 3명 누우면 딱 적당할거 같은 사이즈인거 같아요. 그리고 원래 쓰던 테라6는 접어도 부피가 좀 크고 좀 더 무거웠고, SUV 짐칸에 충분히 실리기는 했습니다. 그에 비해 오토6는 부피도 작고 경차라도 짐칸에 언제든 실어 다닐수있어서 좋습니다. 저같이 귀차니즘에 간편한거 좋아하시면서 미니멀캠핑 하시는분들에게 제격인 텐트같아요. 한번 설치해보면 그 담부터는 아주 쉽습니다. 저처럼 캠핑초보나 나들이용으로 쓰실분들은 저렴하게 사서 쓰시면 좋을거 같네요.'

In [6]:
a = re.sub('[^\uAC00-\uD7A30-9a-zA-Z]', ' ', a)
a

'원래 원터치텐트만 구입하려고 마음먹은 사람입니다  기존 패스트캠프 테라6 원터치 텐트 사용하다가 좀 작은 사이즈도 추가로 구입하고 싶어서 오토6로 하나 더 구입했습니다  손에 익었을때  펴는데 2분정도 걸려요  접는데도 그정도는 걸리는거 같구요 가방에 넣었을때 부피가 작고 가벼워서 좋습니다  미니멀캠핑이라 구성품에 있는 번들팩으로도 사용이 충분하고  추가 팩 구입 할 필요 없을듯해요  나무데크에서 주로 야영하니 오징어팩만 쓸때고 있구요  천고는 테라6가 160이어서 엄청 편했는데 이번에 산 오토6도 그것보다는 낮지만 전혀 불편감 없네요  레인플라이 레인커버   전용카페트  그라운드시트  익스텐션월  추가 구매했습니다  타프 가지고 있는분은 익스텐션월 추가 구매 불필요할거 같구요  딱 비 피하고 짐 안젖을 정도의 공간만 나와요    미니멀캠핑하시는분은 익스텐션월까지 추가 구입하시고  타프 사용 안하시면 될 거 같구요   만원더 보태서 크기가 좀더 큰 착탈익스텐션 월 사시는것도 괜찮을거 같구요   텐트 내부는 어른 3명 누우면 딱 적당할거 같은 사이즈인거 같아요  그리고 원래 쓰던 테라6는 접어도 부피가 좀 크고 좀 더 무거웠고  SUV 짐칸에 충분히 실리기는 했습니다  그에 비해 오토6는 부피도 작고 경차라도 짐칸에 언제든 실어 다닐수있어서 좋습니다  저같이 귀차니즘에 간편한거 좋아하시면서 미니멀캠핑 하시는분들에게 제격인 텐트같아요  한번 설치해보면 그 담부터는 아주 쉽습니다  저처럼 캠핑초보나 나들이용으로 쓰실분들은 저렴하게 사서 쓰시면 좋을거 같네요 '

## 1. cleansing

In [94]:
def review_cleansing(col_review):
    col_review = col_review.str.replace(r'<.+?>', '', regex=True)
    col_review = col_review.str.replace(r'[^\uAC00-\uD7A30-9a-zA-Z]', ' ', regex=True)
    col_review = col_review.str.replace('\s+', ' ', regex=True)
    return col_review

In [112]:
r = review_cleansing(df['review'])
r

0         원래 원터치텐트만 구입하려고 마음먹은 사람입니다 기존 패스트캠프 테라6 원터치 텐트...
1         좋아요 한여름에 창이 4개가 아니라 고생했지많요 확실히 두개라서 바람이 잘안드네요 ...
2         1 배송이 빨리왔어요 2 펼치고 접기 너무 쉬워요 사실 저희부부 둘다 텐트를 펴고 ...
3         구매하기전에 정말 고민 많이했어요 아이와 캠핑가기 위해 설치가 편한 제품을 찾다가결...
4         첫 캠핑이라 너무 설레서 고민고민하다가 무엇을 사야하나 하고 매장도 돌아다니고 고민...
                                ...                        
107274                  배송이 좀 느렸다는점 다음날 받은게 아니어서요 택배사 문제인듯 
107275                                         가격대비 가성비 좋아요
107276                                      상품좋습니다 크기도생각보다큼
107277           색깔이 흰색인줄 알았는데 회색이더군요 아직 펴보진 않았는데 일단 가볍긴하네요
107278                                           만족합니다만족핮니더
Name: review, Length: 107279, dtype: object

In [127]:
#r = r.to_string()

### txt 저장 & 파일 로드

In [130]:
r.to_csv('./tent-review-cleansed.txt', index=False, header=None, sep="\t")
file_name = "./tent-review-cleansed.txt"

with open(file_name, 'r', encoding='utf-8') as f:
    review = f.readlines()

    print(review)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [131]:
review

['원래 원터치텐트만 구입하려고 마음먹은 사람입니다 기존 패스트캠프 테라6 원터치 텐트 사용하다가 좀 작은 사이즈도 추가로 구입하고 싶어서 오토6로 하나 더 구입했습니다 손에 익었을때 펴는데 2분정도 걸려요 접는데도 그정도는 걸리는거 같구요 가방에 넣었을때 부피가 작고 가벼워서 좋습니다 미니멀캠핑이라 구성품에 있는 번들팩으로도 사용이 충분하고 추가 팩 구입 할 필요 없을듯해요 나무데크에서 주로 야영하니 오징어팩만 쓸때고 있구요 천고는 테라6가 160이어서 엄청 편했는데 이번에 산 오토6도 그것보다는 낮지만 전혀 불편감 없네요 레인플라이 레인커버 전용카페트 그라운드시트 익스텐션월 추가 구매했습니다 타프 가지고 있는분은 익스텐션월 추가 구매 불필요할거 같구요 딱 비 피하고 짐 안젖을 정도의 공간만 나와요 미니멀캠핑하시는분은 익스텐션월까지 추가 구입하시고 타프 사용 안하시면 될 거 같구요 만원더 보태서 크기가 좀더 큰 착탈익스텐션 월 사시는것도 괜찮을거 같구요 텐트 내부는 어른 3명 누우면 딱 적당할거 같은 사이즈인거 같아요 그리고 원래 쓰던 테라6는 접어도 부피가 좀 크고 좀 더 무거웠고 SUV 짐칸에 충분히 실리기는 했습니다 그에 비해 오토6는 부피도 작고 경차라도 짐칸에 언제든 실어 다닐수있어서 좋습니다 저같이 귀차니즘에 간편한거 좋아하시면서 미니멀캠핑 하시는분들에게 제격인 텐트같아요 한번 설치해보면 그 담부터는 아주 쉽습니다 저처럼 캠핑초보나 나들이용으로 쓰실분들은 저렴하게 사서 쓰시면 좋을거 같네요 \n',
 '좋아요 한여름에 창이 4개가 아니라 고생했지많요 확실히 두개라서 바람이 잘안드네요 튼튼하고 커서 구매한건데요 피거나 접는건 편해요 몇번해보면 여자도 손쉽게 할수있지만여자가 수레없이 치는장소까지 옮기기엔 무거워요 남자가 들어도 오래들고 이동하긴 무게가있음안에 육각돗자리도샀는데 이건 정말필수인듯해요 꼭 사야될듯합니다 아직 추가구매한 그늘막까진 못쳐봤어요 점점더 쌀쌀해지기전에 놀러가서 사용해볼께요 남편 저 2살아이가 넉넉히 짐넣고 뒹굴뒹굴하기엔 공간 충분

In [118]:
len(review)

107279

In [119]:
review[0][-1]

'\n'

In [120]:
# re.sub(r'\n', '', review[0])

'원래 원터치텐트만 구입하려고 마음먹은 사람입니다 기존 패스트캠프 테라6 원터치 텐트 사용하다가 좀 작은 사이즈도 추가로 구입하고 싶어서 오토6로 하나 더 구입했습니다 손에 익었을때 펴는데 2분정도 걸려요 접는데도 그정도는 걸리는거 같구요 가방에 넣었을때 부피가 작고 가벼워서 좋습니다 미니멀캠핑이라 구성품에 있는 번들팩으로도 사용이 충분하고 추가 팩 구입 할 필요 없을듯해요 나무데크에서 주로 야영하니 오징어팩만 쓸때고 있구요 천고는 테라6가 160이어서 엄청 편했는데 이번에 산 오토6도 그것보다는 낮지만 전혀 불편감 없네요 레인플라이 레인커버 전용카페트 그라운드시트 익스텐션월 추가 구매했습니다 타프 가지고 있는분은 익스텐션월 추가 구매 불필요할거 같구요 딱 비 피하고 짐 안젖을 정도의 공간만 나와요 미니멀캠핑하시는분은 익스텐션월까지 추가 구입하시고 타프 사용 안하시면 될 거 같구요 만원더 보태서 크기가 좀더 큰 착탈익스텐션 월 사시는것도 괜찮을거 같구요 텐트 내부는 어른 3명 누우면 딱 적당할거 같은 사이즈인거 같아요 그리고 원래 쓰던 테라6는 접어도 부피가 좀 크고 좀 더 무거웠고 SUV 짐칸에 충분히 실리기는 했습니다 그에 비해 오토6는 부피도 작고 경차라도 짐칸에 언제든 실어 다닐수있어서 좋습니다 저같이 귀차니즘에 간편한거 좋아하시면서 미니멀캠핑 하시는분들에게 제격인 텐트같아요 한번 설치해보면 그 담부터는 아주 쉽습니다 저처럼 캠핑초보나 나들이용으로 쓰실분들은 저렴하게 사서 쓰시면 좋을거 같네요 '

## 2. 토크나이징

In [4]:
from konlpy.tag import Hannanum, Okt

In [138]:
okt = Okt()
okt.morphs(review[0])

['원래',
 '원',
 '터치',
 '텐트',
 '만',
 '구입',
 '하려고',
 '마음먹은',
 '사람',
 '입니다',
 '기존',
 '패스트',
 '캠프',
 '테라',
 '6',
 '원',
 '터치',
 '텐트',
 '사용',
 '하다가',
 '좀',
 '작은',
 '사이즈',
 '도',
 '추가',
 '로',
 '구입',
 '하고',
 '싶어서',
 '오토',
 '6',
 '로',
 '하나',
 '더',
 '구입',
 '했습니다',
 '손',
 '에',
 '익었을',
 '때',
 '펴는데',
 '2분',
 '정도',
 '걸려요',
 '접는데도',
 '그',
 '정도',
 '는',
 '걸리는거',
 '같구요',
 '가방',
 '에',
 '넣었을',
 '때',
 '부피',
 '가',
 '작고',
 '가벼워서',
 '좋습니다',
 '미니',
 '멀',
 '캠핑',
 '이라',
 '구',
 '성품',
 '에',
 '있는',
 '번',
 '들',
 '팩',
 '으로도',
 '사용',
 '이',
 '충분하고',
 '추가',
 '팩',
 '구입',
 '할',
 '필요',
 '없을듯',
 '해',
 '요',
 '나무',
 '데크',
 '에서',
 '주로',
 '야영',
 '하니',
 '오징어',
 '팩',
 '만',
 '쓸',
 '때',
 '고',
 '있구요',
 '천고',
 '는',
 '테라',
 '6',
 '가',
 '160',
 '이어서',
 '엄청',
 '편했는데',
 '이번',
 '에',
 '산',
 '오토',
 '6',
 '도',
 '그것',
 '보다는',
 '낮',
 '지만',
 '전혀',
 '불편',
 '감',
 '없네요',
 '레인',
 '플라이',
 '레인',
 '커버',
 '전용',
 '카페트',
 '그',
 '라운드',
 '시트',
 '익스',
 '텐션',
 '월',
 '추가',
 '구매',
 '했습니다',
 '타프',
 '가지',
 '고',
 '있는',
 '분',
 '은',
 '익스',
 '텐션',
 '월',
 '추가',
 '구매',
 

In [135]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [139]:
tokenized = []
for sentence in tqdm(review):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized.append(stopwords_removed_sentence)

In [140]:
print(tokenized[:3])

[['원래', '원', '터치', '텐트', '만', '구입', '마음먹다', '사람', '이다', '기존', '패스트', '캠프', '테라', '6', '원', '터치', '텐트', '사용', '작다', '사이즈', '추가', '로', '구입', '하고', '싶다', '오토', '6', '로', '하나', '더', '구입', '손', '익다', '때', '펴다', '2분', '정도', '걸리다', '접다', '그', '정도', '걸리다', '같다', '가방', '넣다', '때', '부피', '작고', '가볍다', '좋다', '미니', '멀다', '캠핑', '이라', '구', '성품', '있다', '번', '팩', '으로도', '사용', '충분하다', '추가', '팩', '구입', '필요', '없다', '해', '요', '나무', '데크', '에서', '주로', '야영', '오징어', '팩', '만', '쓸다', '때', '고', '있다', '천고', '테라', '6', '가다', '160', '이어서', '엄청', '편하다', '이번', '산', '오토', '6', '그것', '보다는', '낮', '지만', '전혀', '불편', '감', '없다', '레인', '플라이', '레인', '커버', '전용', '카페트', '그', '라운드', '시트', '익스', '텐션', '월', '추가', '구매', '타프', '가지', '고', '있다', '분', '익스', '텐션', '월', '추가', '구매', '불필요하다', '같다', '딱', '비', '피하', '고', '짐', '안', '젖다', '정도', '공간', '만', '나오다', '미니', '멀다', '캠핑', '분', '익스', '텐션', '월', '까지', '추가', '구입', '타프', '사용', '안', '되다', '거', '같다', '만', '원더', '보태', '서', '크기', '좀더', '크다', '착탈', '익스', '텐션', '월', '사다', '괜찮다', '같다', '텐트', '내부', 